<a href="https://colab.research.google.com/github/surekhag28/DeepLearning_Projects/blob/master/emotion_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
cd /content/gdrive/My Drive/classifier

/content/gdrive/My Drive/classifier


In [3]:
import pandas as pd

df = pd.read_csv('../data/train_labels.csv')
print(df.head())

   Unnamed: 0                   Image  Category  ...  Fear  Pain  Suffering
0           0  007eear5kx5qhbzewz.jpg         0  ...     0     0          0
1           1  007eear5kx5qhbzewz.jpg         0  ...     0     0          0
2           2  00qbvchuemlwx6muhk.jpg         0  ...     0     0          0
3           3  00ypqjg7ig0ocbl6s2.jpg         0  ...     0     0          0
4           4  0185j147iy2lwsq0p9.jpg         0  ...     0     0          0

[5 rows x 29 columns]


In [9]:
import numpy as np
import torch
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt
import torch.nn as nn
from EmotionDataset import EmotionDataset
from sklearn.metrics import f1_score

NUM_CLASSES = 26
num_epochs = 5

dev = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(dev)

transform = transforms.Compose([transforms.Resize([224,224]),
                                transforms.Grayscale(3),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.5],std=[0.5])
                            ])

test_transform = transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.5],std=[0.5])
                            ])

train_data = EmotionDataset(csv_file = '../data/train_labels.csv',
                                    root_dir = '../data/train/', transform = transform)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=20, shuffle=True, num_workers=1)

val_data = EmotionDataset(csv_file = '../data/val_labels.csv',
                                    root_dir = '../data/val/', transform = transform)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=20, shuffle=True, num_workers=1)

test_data = EmotionDataset(csv_file = '../data/test_labels.csv',
                                    root_dir = '../data/test/', transform = transform)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=20, shuffle=True, num_workers=1)

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet,self).__init__()
        self.num_classes=NUM_CLASSES
        self.conv1 = nn.Conv2d(3,48,kernel_size=5,padding=2)
        self.pool1 = nn.MaxPool2d(kernel_size=(2,2),stride=2)
        self.conv2 = nn.Conv2d(48,64,kernel_size=5,padding=2)
        self.pool2 = nn.MaxPool2d(kernel_size=(2,2),stride=(2,2))
        self.conv3 = nn.Conv2d(64,128,kernel_size=5,padding=2)
        self.pool3 = nn.MaxPool2d(kernel_size=(2,2),stride=(2,2))
        self.fc1 = nn.Linear(28*28*128,2048)
        self.fc2 = nn.Linear(2048,NUM_CLASSES)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self,x):
        x = self.relu(self.pool1(self.conv1(x)))  #input: 224*224
        x = self.relu(self.pool2(self.conv2(x)))  #input 112*112
        x = self.relu(self.pool3(self.conv3(x)))  #input 56*56
        x = x.view(-1,28*28*128)             #28*28
        x = self.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))                            #output: 252
        return x

model = ConvNet()
model = model.to(dev)
#criterion = nn.MultiLabelMarginLoss()

criterion = nn.BCELoss()


optimizer = torch.optim.SGD(model.parameters(),0.1,momentum=0.9)

total_step = len(train_loader)
train_losses, val_losses, train_accs, val_acc, train_precs, val_precs, train_recs, val_recs = [],[],[],[],[],[],[],[]

classes = {0:'Peac' , 1:'Affection',  2:'Esteem', 3: 'Anticipation', 4:'Engagement', 5:'Confidence', 6:'Happiness', 7:'Pleasure', 8:'Excitement', 9:'Surprise', 10:'Sympathy', 11:'Doubt/Confusion', 12:'Disconnection', 13:'Fatigue', 14:'Embarrassment', 15:'Yearning', 16:'Disapproval', 17:'Aversion',  18:'Annoyance', 19:'Anger', 20:'Sensitivity', 21:'Sadness', 22:'Disquietment', 23:'Fear', 24:'Pain', 25:'Suffering'}

correct, pred_positive, pred_negative, class_precision, class_recall  = {},{},{},{},{}

def init():
    for i in range(26):
        correct[i] = 0
        pred_positive[i] = 0
        pred_negative[i] = 0

def calculate_scores(outputs, labels):
    predicted = torch.zeros([20,26])
    init()

    labels = labels.reshape((-1,26))
    for i in range(labels.shape[0]):
        for j in range(labels.shape[1]):
            if (outputs[i][j] > 0.4000):
                predicted[i][j] = 1.
            if (predicted[i][j] == labels[i][j].to('cpu').float()):
                correct[j] += 1
            if (predicted[i][j] == 1. and labels[i][j].to('cpu').float() == 0.):
                pred_positive[j] += 1
            if (predicted[i][j] == 0. and labels[i][j].to('cpu').float() == 1.):
                pred_negative[j] += 1


    return

def calculate_precision():

    for i in range(26):
        if((correct[i] + pred_positive[i]) == 0):
            #print(correct[i] + pred_positive[i])
            continue
        class_precision[i] += correct[i] / (correct[i] + pred_positive[i])
    return class_precision

def calculate_recall():
    for i in range(26):
        if((correct[i] + pred_negative[i]) == 0):
          #print(correct[i] + pred_negative[i])
            continue
        class_recall[i] += correct[i] / (correct[i] + pred_negative[i])
    return class_recall

def mean_precision(precs):
   # for k,v in precs.items():
    print(precs)

for epoch in range(num_epochs):
    train_loss, val_loss, train_acc, val_acc = 0,0,0,0
    for i in range(26):
        class_precision[i] = 0
        class_recall[i] = 0
    for images,labels in train_loader:
        images = images.to(dev)
        labels = labels.to(dev)

        outputs = model(images)
        loss = criterion(outputs, labels.type(torch.FloatTensor).to(dev))
        train_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        calculate_scores(outputs, labels)
        train_prec = calculate_precision()
        train_rec = calculate_recall()
    else:
        with torch.no_grad():
            model.eval()
            for i in range(26):
                class_precision[i] = 0
                class_recall[i] = 0
            for images,labels in val_loader:
                images = images.to(dev)
                labels = labels.to(dev)

                val_out = model(images)
                loss = criterion(val_out, labels.type(torch.FloatTensor).to(dev))
                val_loss += loss.item()

                calculate_scores(val_out, labels)
                val_prec = calculate_precision()
                val_rec = calculate_recall()
        model.train()

        train_prec = {k: round(v / len(train_loader),3) for k, v in train_prec.items()}
        val_prec = {k: round(v / len(val_loader),3) for k, v in val_prec.items()}
        train_rec = {k: round(v / len(train_loader),3) for k, v in train_rec.items()}
        val_rec = {k: round(v / len(val_loader),3) for k, v in val_rec.items()}
    print("Epoch: {}/{}.. ".format(epoch, num_epochs),
              "Training Loss: {:.3f}.. ".format(train_loss/len(train_loader)),
              "Validation Loss: {:.3f}.. ".format(val_loss/len(val_loader)))

    #print("Class wise precision for training set: {}".format(train_prec))
    #print("Class wise precision for validation set: {}".format(val_prec))
    #print("Class wise recall for training set: {}".format(train_rec))
    #print("Class wise recall for validation set: {}".format(val_rec))
    train_losses.append(train_loss/len(train_loader))
    val_losses.append(val_loss/len(val_loader))
    train_precs.append(train_prec)
    val_precs.append(val_prec)
    train_recs.append(train_rec)
    val_recs.append(val_rec)

    #mean_precision(val_precs)
model.eval()
with torch.no_grad():
    for i in range(26):
        class_precision[i] = 0
        class_recall[i] = 0
    for images, labels in test_loader:
        images = images.to(dev)
        labels = labels.to(dev)

        outputs = model(images)

        calculate_scores(outputs, labels)
        test_prec = calculate_precision()
        test_rec = calculate_recall()
    test_prec = {k: round(v / len(test_loader),3) for k, v in test_prec.items()}
    test_rec = {k: round(v / len(test_loader),3) for k, v in test_rec.items()}

print('Test precision of the model on the 10000 test images: ', test_prec)
print('Test recall of the model on the 10000 test images: ', test_rec)
#print(mean_precision(test_prec))        
                            

cuda:0


FileNotFoundError: ignored